In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import yfinance as yf
import dash
from dash import dcc
from dash import html

## Importing data sample containing ESG scores and Adj Close prices of 451 SP500 companies

In [2]:
df_data = pd.read_excel('sp500_companies.xlsx', header=[0,1], index_col=0)

## Filtering stocks based on ESG score

In [3]:
from filter_sp500_stocks import filter_esg_stocks
filtered = filter_esg_stocks(df_data, 'ESG', threshold = 0.7)

## Creating the investors portfolio

In [4]:
from create_portfolio import create_portfolio1
selected_portfolio = create_portfolio1(filtered)

IntSlider(value=1, continuous_update=False, description='Number of stocks:', max=451, min=1)

Dropdown(description='Stocks:', layout=Layout(width='300px'), options=('A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC…

Button(description='Add stock', style=ButtonStyle())

Button(description='Exit', style=ButtonStyle())

Button(description='Reset portfolio', style=ButtonStyle())

Output()

Stock A has been added to your portfolio
Stock AAL has been added to your portfolio
Stock AAP has been added to your portfolio
Stock AAPL has been added to your portfolio
Stock ABBV has been added to your portfolio
Stock ABC has been added to your portfolio
Stock ABT has been added to your portfolio
Stock ACN has been added to your portfolio


Environmental       Social       Governance       ESG   
     Environmental Score Social Score Governance Score ESG Score   
A               0.433930     0.178957         0.237553  0.850440  \
AAL             0.373963     0.261069         0.225789  0.860821   
AAP             0.280258     0.340479         0.243199  0.863935   
AAPL            0.344175     0.272432         0.247224  0.863831   
ABBV            0.386143     0.235547         0.237864  0.859554   
ABC             0.404483     0.231658         0.220627  0.856768   
ABT             0.364772     0.249027         0.248326  0.862126   
ACN             0.396327     0.221804         0.239882  0.858012   

               Adj Close                                           
     2014-12-31 00:00:00 2015-12-31 00:00:00 2016-12-31 00:00:00   
A              38.084961           39.297142           43.283852  \
AAL            50.814602           40.471584           45.118084   
AAP           147.069260          139.179230          156.636154   
AAPL           24.801630           24.054173           27.056238   
ABBV           45.798306           42.821068           47.014099   
ABC            78.874535           91.799034           70.389420   
ABT            38.343544           39.061234           34.248650   
ACN            77.527664           92.668983          105.968575   

                                                                   
     2017-12-31 00:00:00 2018-12-31 00:00:00 2019-12-31 00:00:00   
A              64.241348           65.294334           83.306152  \
AAL            50.711884           31.599043           28.574404   
AAP            92.502930          146.388016          149.126236   
AAPL           40.168854           38.003681           71.810944   
ABBV           75.257782           74.537941           75.632286   
ABC            84.124290           69.344009           80.772385   
ABT            52.067604           67.199394           82.037163   
ACN           141.294174          132.493439          200.348068   

                                              
     2020-12-31 00:00:00 2021-12-31 00:00:00  
A             116.700371          158.140182  
AAL            15.770000           17.959999  
AAP           147.615494          227.725098  
AAPL          130.916138          176.276230  
ABBV           96.596252          127.918587  
ABC            94.530449          130.465134  
ABT           105.040977          137.108215  
ACN           252.519485          405.711395

## Calculating log returns for the selected stocks

In [5]:
from log_return import log_returns
log_re = log_re = log_returns(selected_portfolio['Adj Close']).replace([np.inf, -np.inf], np.nan).dropna(axis=1).T

## Storing ESG, E, S and G scores in a numpy array

In [6]:
esg = selected_portfolio['ESG']['ESG Score']
esg_np = esg.to_numpy()

e = selected_portfolio['Environmental']['Environmental Score']
e_np = e.to_numpy()
s = selected_portfolio['Social']['Social Score']
s_np = s.to_numpy()
g = selected_portfolio['Governance']['Governance Score']
g_np = g.to_numpy()

## Calculating the Covariance matrix, inverse covariance matrix and mean return for the stocks

In [7]:
from covariance import cov_matrix
from mean_ret import mean_return

cov_var = cov_matrix(log_re)
inv_cov = np.linalg.inv(cov_var)
mean_re = mean_return(log_re)

## Finding two optimal portfolios with different expected return

In [8]:
from ones import vector_ones
from mvp import mvp_func
from weights_mvp import mvp_weights

labels = selected_portfolio.index
vector_one = vector_ones(labels)
a, b, c, d = mvp_func(inv_cov, mean_re, vector_one)
w1, w2 = mvp_weights(a, b, c, d, inv_cov, vector_one, mean_re, 0.05, 0.18)

## Two optimal portfolios are now obtained. Thus, we can use the two fund theorem to find the weights for the entire efficient frontier. Hereby, we also compute the portfolios expected returns, volatility and weighted avereage esg score

In [9]:
from alpha import alpha
from two_fun_theorem import w3

alpha_list = alpha([-1])
ef_weights, r, vol, esg_wa = w3(w1, w2, alpha_list, mean_re, cov_var, esg_np)
len(ef_weights)

1001

### Tickerlist for the plot

In [10]:
tickers_list = []
for i in range(len(ef_weights)):
    #assets = np.random.choice(list(labels), len(labels), replace=False)
    assets = list(labels)
    tickers_list.append(assets)
sr_t_list = list(labels)

## Obtaining the global MVP. Furthermore, we calculate the expected return, volatility and WA ESG score for the portfolio

In [11]:
from global_mvp import global_mvp
mvp_weight, mvp_er, mvp_vol, mvp_esg_score = global_mvp(ef_weights, vol, mean_re, cov_var, esg_np)

## Indicators for the ESG investor - from paper written by Bastien Drut

In [12]:
from esg_indicators import indicators
indicators(vector_one, inv_cov, mean_re, r, esg_np, 0.8635)

Variable outputs used from the paper
------------------------------------
the value of delta 0 is: 0.8602685510191886

-------------------------------------
Is delta 1 larger than zero: False
-------------------------------------
The value of delta 1 is: -0.004829253105757657

-------------------------------------
The value of Lambda 0 is: -3.1559744178169744

-------------------------------------
The value of gamma 0 is: 0.10741172416610167

-------------------------------------
The value of gamma 1 is: 2.4507790470081856

-------------------------------------
The value of gamma 2 is: 0.011582505135920792

--------------------------------------
Is MVP ESG rating higher than ESG threshold: False
--------------------------------------
the ESG score of the MVP is: 0.8596891763598657

--------------------------------------
the WA ESG score for traditional investor portfolios is: [0.85877148 0.85877776 0.85878404 ... 0.86503696 0.86504323 0.86504951]


## Deriving the optimal portfolios for the ESG investor. We furthermore derive the expected return, volatility and WA ESG score for the ESG investors efficient frontier

In [13]:
from esg_investor_weights import minimize_func
list_target_returns = np.arange(r.min(), r.max(), 0.0004).tolist()
esg_investor_weights, esg_portfolio_returns, esg_portfolio_vol, esg_investor_wa  = minimize_func(mean_re, cov_var, esg_np, 0.8603, list_target_returns, mvp_weight)

## Sharpe Ratio for the traditional investor and ESG investor

In [16]:
from max_sr import sharpe_ratio
sr, optimal_sharpe_ratio, w_sr, sr_er, vol_sr, sr_score, sr_e_score, sr_s_score, sr_g_score, sr_esg, optimal_sharpe_ratio_esg, w_sr_esg, sr_er_esg, vol_sr_esg, sr_esg_score, sr_esg_e_score, sr_esg_s_score, sr_esg_g_score= sharpe_ratio(0.2, mean_re, cov_var, ef_weights, r, vol, esg_investor_weights, esg_portfolio_returns, esg_portfolio_vol, esg_np, e_np, s_np, g_np)

In [17]:
optimal_sharpe_ratio_esg

0.7280031221234303

In [18]:
optimal_sharpe_ratio

0.7398975566124063

## Sortino Ratio for the traditional investor and ESG investor

In [19]:
from sortino import sortino
sortino_ratio, w_sortino, optimal_sortino_ratio, sortino_exp_return, sortino_vol, sortino_ratio_esg, w_sortino_esg, optimal_sortino_ratio_esg, sortino_exp_return_esg, sortino_vol_esg = sortino(0.03, mean_re, cov_var, ef_weights, r, esg_investor_weights, esg_portfolio_returns, esg_np, e_np, s_np, g_np)

In [20]:
optimal_sharpe_ratio

0.7398975566124063

In [21]:
optimal_sharpe_ratio_esg

0.7280031221234303

## Plotting the efficient frontier

In [22]:
from plot_ef import ef_plot
plot = ef_plot(tickers_list, sr_t_list, 
               ef_weights, vol, r, esg_wa, 
               esg_investor_weights, esg_portfolio_vol, esg_portfolio_returns, esg_investor_wa, 
               w_sr, vol_sr, sr_er, optimal_sharpe_ratio, sr_score,
               w_sr_esg, vol_sr_esg, sr_er_esg, optimal_sharpe_ratio_esg, sr_esg_score,
               mvp_weight, mvp_vol, mvp_er, mvp_esg_score)

plot.show()

In [23]:
#app = dash.Dash()
#app.layout = html.Div([
 #   dcc.Graph(figure=plot)
#])

#app.run_server(debug=True, port=8052, use_reloader=False)

In [24]:
from rolling_functions import rolling_ef
ret_mean_rolling, cov_rolling, list_ef_weights, list_exp_return, list_volatility, wa_esg_score, list_wa_e_score, list_wa_s_score, list_wa_g_score, list_mvp_weights, list_mvp_exp_return, list_mvp_volatility, list_mvp_esg, list_esg_ef_weights, list_target_returns, list_esg_exp_return, list_esg_variance, list_esg_volatility, list_esg_esg_score, list_esg_e_score, list_esg_s_score, list_esg_g_score, list_sr, list_sr_opt, list_sr_esg, list_sr_esg_opt, list_sr_w, list_sr_w_esg, list_sr_er, list_sr_esg_er, list_sr_vol, list_sr_esg_vol, list_sr_score, list_sr_e_score, list_sr_s_score, list_sr_g_score, list_sr_esg_score, list_sr_esg_e_score, list_sr_esg_s_score, list_sr_esg_g_score = rolling_ef(log_re, 3, 0.02, esg_np, e_np, s_np, g_np, 0.8)

In [33]:
list_sr_er

[0.46, 0.3429999999999998, 0.4276, 0.4599999999999999, 0.3574000000000001]

In [34]:
list_sr_esg_er

[0.4597000000138689,
 0.3424999999929527,
 0.4273000000557199,
 0.4596999999955854,
 0.35810000002397924]

In [25]:
list_sr_score

[0.8520292887816515,
 0.8601306386009533,
 0.8597033675472984,
 0.8625075831963377,
 0.859359688595944]

In [26]:
list_sr_e_score

[0.4332519219540071,
 0.3545814278301384,
 0.36731085792312096,
 0.3233711370106232,
 0.37455335479088364]

In [27]:
list_sr_s_score

[0.16269126765442396,
 0.2470348359118629,
 0.2385231520452821,
 0.28597956139972164,
 0.2435948987734168]

In [28]:
list_sr_g_score

[0.25608609917322056,
 0.2585143748589519,
 0.25386935757889517,
 0.25315688478599263,
 0.24121143503164338]

In [29]:
list_sr_esg_score

[0.8520351031383893,
 0.860130500040363,
 0.859703258495712,
 0.8625056301080328,
 0.8593612787643385]

In [30]:
list_sr_esg_e_score

[0.43322569241017495,
 0.3545914991024461,
 0.3673055589068308,
 0.3233996640045377,
 0.374523952300053]

In [31]:
list_sr_esg_s_score

[0.16274374729903168,
 0.24706162468436133,
 0.23854155928679377,
 0.28596105612885664,
 0.2436084049069024]

In [32]:
list_sr_esg_g_score

[0.2560656634291827,
 0.2584773762535554,
 0.25385614030208725,
 0.2531449099746385,
 0.2412289215573831]

In [35]:
df_eco = pd.DataFrame()

In [37]:
df_eco['E Score_sens'] =  list_sr_esg_e_score
df_eco['S Score_sens'] =  list_sr_esg_s_score
df_eco['G Score_sens'] =  list_sr_esg_g_score
df_eco['ESG Score_sens'] = list_sr_esg_score
df_eco['E Score_trad'] =  list_sr_e_score
df_eco['S Score_trad'] =  list_sr_s_score
df_eco['G Score_trad'] =  list_sr_g_score
df_eco['ESG Score_trad'] = list_sr_score
df_eco['Return_sens'] = list_sr_esg_er
df_eco['Return_trad'] = list_sr_er
df_eco['Rolling_id'] = [1,2,3,4,5]
df_eco['Investor_type'] = []

In [38]:
df_eco

E Score_sens  S Score_sens  G Score_sens  ESG Score_sens  E Score_trad   
0      0.433226      0.162744      0.256066        0.852035      0.433252  \
1      0.354591      0.247062      0.258477        0.860131      0.354581   
2      0.367306      0.238542      0.253856        0.859703      0.367311   
3      0.323400      0.285961      0.253145        0.862506      0.323371   
4      0.374524      0.243608      0.241229        0.859361      0.374553   

   S Score_trad  G Score_trad  ESG Score_trad  Return_sens  Return_trad   
0      0.162691      0.256086        0.852029       0.4597       0.4600  \
1      0.247035      0.258514        0.860131       0.3425       0.3430   
2      0.238523      0.253869        0.859703       0.4273       0.4276   
3      0.285980      0.253157        0.862508       0.4597       0.4600   
4      0.243595      0.241211        0.859360       0.3581       0.3574   

   Rolling_id  
0           1  
1           2  
2           3  
3           4  
4           5